In [1]:
import sentencepiece as spm
import pandas as pd

In [10]:
data = pd.read_csv("C:/Users/rltmdals/Desktop/botchan.txt", sep='\t', quoting=3)

In [11]:
print(data.shape)
data.head()

(4287, 1)


,"Project Gutenberg's Botchan (Master Darling), by Kin-nosuke Natsume"
0,This eBook is for the use of anyone anywhere a...
1,almost no restrictions whatsoever. You may co...
2,re-use it under the terms of the Project Guten...
3,with this eBook or online at www.gutenberg.org
4,Title: Botchan (Master Darling)


In [4]:
input_file = "C:/Users/rltmdals/Desktop/botchan.txt"
vocab_size = 2000
prefix = './mss'

In [5]:
templates = '--input={} --model_prefix={} --vocab_size={}'
cmd = templates.format(input_file, prefix, vocab_size)
spm.SentencePieceTrainer.Train(cmd)

In [6]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [12]:
# 학습이 얼마나 잘됬는지 확인
# encode: text => id
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁is', '▁a', '▁t', 'est']
[212, 32, 10, 587, 446]


In [20]:
print(sp.encode_as_pieces('The ablest can do no more, and to want more than this will be demanding something impossible.'))

['▁the', '▁able', 'st', '▁can', '▁do', '▁no', '▁more', ',', '▁and', '▁to', '▁want', '▁more', '▁than', '▁this', '▁will', '▁be', '▁demand', 'ing', '▁something', '▁imp', 'o', 's', 's', 'ible', '.']


In [15]:
# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids([209, 31, 9, 375, 586]))
# print(sp.decode_ids([212, 32, 10, 587, 446]))

This is a test
okn  k star


## Text normalization
Sentencepiece provides the following general pre-defined normalization rules. We can change the normalizer with --normaliation_rule_name=<NAME> flag.

* nmt_nfkc: NFKC normalization with some additional normalization around spaces. (default)
* nfkc: original: NFKC normalization.
* nmt_nfkc_cf: nmt_nfkc + Unicode case folding (mostly lower casing)
* nfkc_cf: nfkc + Unicode case folding.
* identity: no normalization

In [18]:
# NFKC normalization and lower casing.
spm.SentencePieceTrainer.train('--input=C:/Users/rltmdals/Desktop/botchan.txt --model_prefix=m --vocab_size=2000 --normalization_rule_name=nfkc_cf')

sp = spm.SentencePieceProcessor()
sp.load('m.model')
print(sp.encode_as_pieces('ＨＥＬＬＯ　ＷＯＲＬＤ.'))  # lower casing and normalization
print(sp.encode_as_pieces('T H I S I S A T E S T.'))

['▁', 'hello', '▁world', '.']
['▁t', '▁', 'h', '▁i', '▁s', '▁i', '▁s', '▁a', '▁t', '▁e', '▁s', '▁t', '.']


In [21]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [22]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x189f8aa3220>)

In [23]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [24]:
train_df.shape
# 리뷰 개수 5만개

(50000, 2)

In [25]:
# 총 5만개의 샘플이 존재합니다. 센텐스피스의 입력으로 사용하기 위해서 데이터프레임을 txt 파일로 저장합니다.
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [26]:
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')